In [1]:
import numpy as np
from numpy.typing import NDArray

# 1) Generate ellipse points
n: int = 800
a: float = 5.0
b: float = 2.0
angle_deg: float = 25.0
noise_std: float = 0.25
seed: int = 42

rng = np.random.default_rng(seed)
t: NDArray[np.floating] = rng.uniform(0.0, 2.0 * np.pi, size=n)

base: NDArray[np.floating] = np.stack([a * np.cos(t), b * np.sin(t)], axis=0)  # (2, n)
angle: float = float(np.deg2rad(angle_deg))
r: NDArray[np.floating] = np.array(
    [[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]], dtype=float
)

pts: NDArray[np.floating] = (r @ base).T  # (n, 2)
pts = pts + rng.normal(0.0, noise_std, size=pts.shape)

x: NDArray[np.floating] = pts[:, 0]
y: NDArray[np.floating] = pts[:, 1]


In [2]:
import numpy as np
from numpy.typing import NDArray

# 2) Covariance + eigenvectors
center_x: float = float(np.mean(x))
center_y: float = float(np.mean(y))

x0: NDArray[np.floating] = x - center_x
y0: NDArray[np.floating] = y - center_y
z: NDArray[np.floating] = np.stack([x0, y0], axis=0)  # (2, n)

cov: NDArray[np.floating] = (z @ z.T) / (n - 1)

eigvals: NDArray[np.floating]
eigvecs: NDArray[np.floating]
eigvals, eigvecs = np.linalg.eigh(cov)  # eigvals asc

order: NDArray[np.integer] = np.argsort(eigvals)[::-1]
eigvals = eigvals[order]
eigvecs = eigvecs[:, order]

cov, eigvals, eigvecs


(array([[10.50643396,  4.02288427],
        [ 4.02288427,  3.97972036]]),
 array([12.42314429,  2.06301003]),
 array([[-0.90276893,  0.43012586],
        [-0.43012586, -0.90276893]]))

In [3]:
import numpy as np
from bokeh.plotting import figure, output_notebook, show

# 3) Bokeh visualization
output_notebook()

scale: float = 2.0
lengths = scale * np.sqrt(np.maximum(eigvals, 0.0))
v1 = eigvecs[:, 0] * lengths[0]
v2 = eigvecs[:, 1] * lengths[1]

p = figure(width=700, height=700, match_aspect=True, title="Eigenvectors are ellipse axes")
p.circle(x, y, size=4, alpha=0.6)

cx, cy = center_x, center_y
p.line([cx - v1[0], cx + v1[0]], [cy - v1[1], cy + v1[1]], line_width=3, color="red", legend_label="v1")
p.line(
    [cx - v2[0], cx + v2[0]],
    [cy - v2[1], cy + v2[1]],
    line_width=3,
    color="purple",
    legend_label="v2",
)

tt = np.linspace(0.0, 2.0 * np.pi, 256)
circle = np.stack([np.cos(tt), np.sin(tt)], axis=0)
stretch = np.diag(lengths)
curve = eigvecs @ (stretch @ circle)

p.line(curve[0, :] + cx, curve[1, :] + cy, line_width=2, color="green", alpha=0.9, legend_label="ellipse from cov")

p.legend.location = "top_left"
show(p)


Loading BokehJS ...